<a href="https://colab.research.google.com/github/trajinthan/Creditcard-Default-prediction/blob/master/day1_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd   #to use read_csv(), crosstab()
import numpy as np    #to use exponentiation : exp()
import statsmodels.api as sm  #to use the Logit()
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#import the dataset
traindata=pd.read_csv('credit_card_default_train.csv',sep=",")
testdata=pd.read_csv('credit_card_default_test.csv',sep=",")



In [0]:
traindata.head()

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
0,A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
1,A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
2,A4,100K,F,High School,Single,31-45,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
3,A5,200K,F,Graduate,Single,31-45,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
4,A6,1M,F,Graduate,Other,31-45,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [0]:
traindata.describe()

,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,24000.000000,2.400000e+04,24000.000000
mean,-0.014208,-0.130042,-0.158667,-0.215542,-0.261167,-0.287667,1.188697e+05,1.140727e+05,1.092435e+05,1.003573e+05,9.377699e+04,9.034068e+04,1.330648e+04,1.386675e+04,1.209312e+04,1.122527e+04,11174.737208,1.230122e+04,0.222083
std,1.124897,1.200566,1.198875,1.166608,1.131855,1.147847,1.710424e+05,1.652414e+05,1.614737e+05,1.490071e+05,1.417389e+05,1.387237e+05,3.935742e+04,5.648588e+04,4.065924e+04,3.624693e+04,36040.821759,4.233848e+04,0.415656
min,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-3.824900e+05,-1.611850e+05,-1.420790e+05,-3.927000e+05,-1.878820e+05,-7.844830e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,8.246250e+03,6.968750e+03,6.238500e+03,5.429000e+03,4.179750e+03,2.913000e+03,2.310000e+03,1.956000e+03,9.010000e+02,6.930000e+02,610.000000,3.070000e+02,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.156850e+04,4.871700e+04,4.641150e+04,4.410500e+04,4.186300e+04,3.940900e+04,4.920000e+03,4.646500e+03,4.197000e+03,3.465000e+03,3465.000000,3.465000e+03,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.562745e+05,1.489050e+05,1.401622e+05,1.269750e+05,1.169262e+05,1.144352e+05,1.160500e+04,1.155000e+04,1.062600e+04,9.360000e+03,9411.500000,9.252500e+03,0.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,2.228020e+06,2.272881e+06,3.844046e+06,2.059564e+06,2.141765e+06,2.221444e+06,2.017905e+06,3.890638e+06,2.069852e+06,1.434510e+06,965557.000000,1.221218e+06,1.000000


In [0]:
# result = traindata.dtypes 
  
# # Print the result 
# print(result) 

In [0]:
traindata=traindata.replace(['M','F'], [1,-1]) 

In [0]:
pd.to_numeric(traindata['Gender'])
# result = traindata.dtypes 
  
# # Print the result 
# print(result) 

0        1
1       -1
2       -1
3       -1
4       -1
        ..
23995   -1
23996   -1
23997   -1
23998   -1
23999   -1
Name: Gender, Length: 24000, dtype: int64

In [0]:
print(traindata.groupby('EDUCATION_STATUS').count())

                  Client_ID  Balance_Limit_V1  ...  PAID_AMT_DEC  NEXT_MONTH_DEFAULT
EDUCATION_STATUS                               ...                                  
Graduate               8478              8478  ...          8478                8478
High School            3925              3925  ...          3925                3925
Other                 11597             11597  ...         11597               11597

[3 rows x 24 columns]


In [0]:
traindata=traindata.replace(['Graduate','High School','Other'], [2,1,0]) 

In [0]:
pd.to_numeric(traindata['EDUCATION_STATUS'])
# result = traindata.dtypes 
  
# # Print the result 
# print(result)

0        2
1        1
2        1
3        2
4        2
        ..
23995    1
23996    1
23997    0
23998    1
23999    0
Name: EDUCATION_STATUS, Length: 24000, dtype: int64

In [0]:
print(traindata.groupby('MARITAL_STATUS').count())

                Client_ID  Balance_Limit_V1  ...  PAID_AMT_DEC  NEXT_MONTH_DEFAULT
MARITAL_STATUS                               ...                                  
0                   13070             13070  ...         13070               13070
Single              10930             10930  ...         10930               10930

[2 rows x 24 columns]


In [0]:
traindata=traindata.replace(['Single'], [1]) 

In [0]:
pd.to_numeric(traindata['EDUCATION_STATUS'])
# result = traindata.dtypes 
  
# # Print the result 
# print(result)

0        2
1        1
2        1
3        2
4        2
        ..
23995    1
23996    1
23997    0
23998    1
23999    0
Name: EDUCATION_STATUS, Length: 24000, dtype: int64

In [0]:
print(traindata.groupby('AGE').count())

              Client_ID  Balance_Limit_V1  ...  PAID_AMT_DEC  NEXT_MONTH_DEFAULT
AGE                                        ...                                  
31-45             12124             12124  ...         12124               12124
46-65              4150              4150  ...          4150                4150
Less than 30       7638              7638  ...          7638                7638
More than 65         88                88  ...            88                  88

[4 rows x 24 columns]


In [0]:
traindata['AGE']=traindata['AGE'].replace(['Less than 30','31-45','46-65','More than 65'], [1,2,3,4]) 

In [0]:
# result = traindata.dtypes 
  
# # Print the result 
# print(result)

In [0]:
traindata['Balance_Limit_V1'] = traindata['Balance_Limit_V1'].map(lambda x : float(x[0:-1])*1000 if (x[-1]=='K') else float(x[0:-1])*1000000)

In [0]:
pd.to_numeric(traindata['Balance_Limit_V1'])
# result = traindata.dtypes 
  
# # Print the result 
# print(result)

0        1000000.0
1        1000000.0
2         100000.0
3         200000.0
4        1000000.0
           ...    
23995    1000000.0
23996     400000.0
23997     100000.0
23998     200000.0
23999     200000.0
Name: Balance_Limit_V1, Length: 24000, dtype: float64

In [0]:
print(traindata.groupby('NEXT_MONTH_DEFAULT').count())

                    Client_ID  Balance_Limit_V1  ...  PAID_AMT_NOV  PAID_AMT_DEC
NEXT_MONTH_DEFAULT                               ...                            
0                       18670             18670  ...         18670         18670
1                        5330              5330  ...          5330          5330

[2 rows x 24 columns]


In [0]:
# default = traindata['NEXT_MONTH_DEFAULT'] == 1
# df_try = traindata[default]
# traindata.append([df_try]*2,ignore_index=True)

In [0]:
traindata[traindata.columns[12:24]]=traindata[traindata.columns[12:24]]/traindata.max()[traindata.columns[12:24]]

In [0]:
traindata['Balance_Limit_V1']=traindata['Balance_Limit_V1']/traindata.max()['Balance_Limit_V1']

In [0]:
traindata['BIAS']=1

In [0]:
independentVar= ['Balance_Limit_V1',
                 'Gender',
                 'EDUCATION_STATUS',
                 'MARITAL_STATUS',
                 'AGE',
                 'PAY_JULY','PAY_AUG','PAY_SEP','PAY_OCT','PAY_NOV','PAY_DEC',
                 'DUE_AMT_JULY','DUE_AMT_AUG','DUE_AMT_SEP','DUE_AMT_OCT','DUE_AMT_NOV','DUE_AMT_DEC',
                 'PAID_AMT_JULY','PAID_AMT_AUG','PAID_AMT_SEP','PAID_AMT_OCT','PAID_AMT_NOV','PAID_AMT_DEC','BIAS']

X_train, y_train=traindata[independentVar],traindata['NEXT_MONTH_DEFAULT']

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


# define oversampling strategy
# over = RandomOverSampler(sampling_strategy=0.1)
# # fit and apply the transform
# X_train_res, y_train_res = over.fit_resample(X, y)
# define undersampling strategy
under = RandomUnderSampler(sampling_strategy=0.4)
# fit and apply the transform
X_train_res, y_train_res = under.fit_resample(X_train, y_train)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train_res, y_train_res.ravel())

from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf', C=17, gamma=0.6,coef0=2)
# svclassifier=SVC(kernel='poly', C=100, gamma='auto', degree=3,coef0=1)
svclassifier.fit(X_train_res, y_train_res)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


SVC(C=17, break_ties=False, cache_size=200, class_weight=None, coef0=2,
    decision_function_shape='ovr', degree=3, gamma=0.6, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_pred = svclassifier.predict(X_train)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_train, y_pred)
print(confusion_matrix)

[[16153  2517]
 [ 1457  3873]]


In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.87      0.89     18670
           1       0.61      0.73      0.66      5330

    accuracy                           0.83     24000
   macro avg       0.76      0.80      0.78     24000
weighted avg       0.85      0.83      0.84     24000

